In [2]:
import re
import pymongo
import pandas as pd
from collections import Counter
import math
from normalization import  normalize_corpus
from mgp import MovieGroupProcess
from tfidf import getKeywords_tfidf
import sys,codecs

In [4]:
#读取数据
client = pymongo.MongoClient('10.245.142.249', 27017)
# 连接所需数据库、集合,twitter_search为数据库名，mingyan为表名
mongodata = client['twitter_search']['user_timeline'].find()
twitter_data=[]
for item in mongodata:
    t1=item['user_post']['full_text'] 
    data={}
    data['_id']=str(item["_id"])
    data['fulltext']=t1
    twitter_data.append(data)
#将twitter数据转换为Dataframe数据格式    
twitter_data=pd.DataFrame(twitter_data)

#分词去停用词等文本处理
documents,cleaned_fulltext=normalize_corpus(twitter_data)
print(documents[:10])
print(cleaned_fulltext[:10])

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.712 seconds.
Prefix dict has been built succesfully.


['感觉 威高 广场 见', 'Soo Cute', '弱', '眞 白 茉玖', 'OVA 注文 Sing For You 発 売 周年 记念 月 19 日 渋 谷 月 20 日 注文 OIOI 开 催 决定 描 使用 贩 売 展示 盛', '中二恋 诞生 日 凸守 络 最高 尊 丹生 谷森夏生 诞祭 2019 月 30 日 丹生 谷森夏 诞生 日 中二病 恋 祝 全员', '感觉 川普靠 炒股 应该 赚 不少 钱', '头发', '乃木 园子 生诞 祭 乃木 园子 生诞 祭 2019', '指']
['感觉在威高广场见过你', 'Soo Cute', '弱', '眞白 茉玖 。', 'OVA 注文 Sing For You 発売 5周年 记念 9月19日 渋谷 9月20日 注文 OIOI 开催 决定 描 使用 贩売 展示 盛', '中二恋 一番好 诞生日 凸守 络 最高 尊 丹生谷森夏生诞祭2019 8月30日 丹生谷森夏 诞生日 中二病 恋 祝 人  人全员', '我感觉川普靠炒股应该赚了不少钱', '这头发', '乃木园子生诞祭 乃木园子生诞祭2019', '指']


In [5]:
#进行迭代采样聚类
def compute_V(texts):
        V = set()
        for text in texts:
            for word in text:
                V.add(word)
        return len(V)
    
    
texts = [text.split() for text in documents]
V = compute_V(texts)
mgp=MovieGroupProcess(K=6,alpha=0.01,beta=0.02,n_iters=40)
y=mgp.fit(texts,V)

In stage 0: transferred 682 clusters with 6 clusters populated
In stage 1: transferred 427 clusters with 6 clusters populated
In stage 2: transferred 263 clusters with 6 clusters populated
In stage 3: transferred 169 clusters with 6 clusters populated
In stage 4: transferred 136 clusters with 6 clusters populated
In stage 5: transferred 108 clusters with 6 clusters populated
In stage 6: transferred 102 clusters with 6 clusters populated
In stage 7: transferred 90 clusters with 6 clusters populated
In stage 8: transferred 92 clusters with 6 clusters populated
In stage 9: transferred 96 clusters with 6 clusters populated
In stage 10: transferred 83 clusters with 6 clusters populated
In stage 11: transferred 76 clusters with 6 clusters populated
In stage 12: transferred 67 clusters with 6 clusters populated
In stage 13: transferred 65 clusters with 6 clusters populated
In stage 14: transferred 64 clusters with 6 clusters populated
In stage 15: transferred 65 clusters with 6 clusters popul

In [6]:
print(y[:10])
print(Counter(y))
twitter_data['cleaned_fulltext']=cleaned_fulltext
twitter_data['cluster']=y#把得到的类别号存到对应的文章上
twitter_data.head()

[4, 4, 0, 0, 1, 1, 3, 0, 1, 2]
Counter({0: 664, 5: 341, 3: 247, 1: 206, 4: 181, 2: 163})


,_id,fulltext,cleaned_fulltext,cluster
0,5d68f178047c01f0a519557b,@HHuoyanlai 感觉在威高广场见过你,感觉在威高广场见过你,4
1,5d68f17d047c01f0a5195a12,RT @KawaiiBOTz: Soo Cute 😳\n\n🎁 @AceCatgirlBot...,Soo Cute,4
2,5d68f17d047c01f0a5195a14,RT @nEdia_R: 「ここが弱いのか～？」 https://t.co/W4VYnrxcSl,弱,0
3,5d68f17d047c01f0a5195a16,RT @capri134: 眞白いこね＆茉玖ろいこ。 https://t.co/PdInI1...,眞白 茉玖 。,0
4,5d68f17d047c01f0a5195a18,RT @usagi_anime: OVA「ご注文はうさぎですか？？ ～Sing For Yo...,OVA 注文 Sing For You 発売 5周年 记念 9月19日 渋谷 9月20日 注...,1


In [7]:
def get_cluster_data(twitter_data,num_clusters):
    cluster_details={}
    #获取蔟的中心
   
    #获取每个簇的关键特征
    #获取每个簇的id
    for cluster_num in range(num_clusters):
        cluster_details[cluster_num]={}
        cluster_details[cluster_num]['cluster_num']=cluster_num#簇序号
        
        id_s=twitter_data[twitter_data['cluster']==cluster_num]['_id'].values.tolist()
        cluster_details[cluster_num]['_id']=id_s
        
        fulltext=twitter_data[twitter_data['cluster']==cluster_num]['cleaned_fulltext'].values.tolist()
        cluster_details[cluster_num]['Cleaned_fulltext']=fulltext
    
    return cluster_details

def print_cluster_data(cluster_data):
    #print cluster details
    for cluster_num,cluster_details in cluster_data.items():
        print('Cluster{} details'.format(cluster_num))
        print('-'*20)
#         print('Key features:',cluster_details['key_features'])
        print('sentence in this clusters:')
        print('》'.join(cluster_details['Cleaned_fulltext']).replace('\n',''))
        print('='*40)

In [8]:
#打印输出聚类结果文本
num_clusters=len(set(y))

cluster_data=get_cluster_data(twitter_data,num_clusters)
print_cluster_data(cluster_data)

Cluster0 details
--------------------
sentence in this clusters:
弱》眞白 茉玖 。》这头发》好想看点兔OVA，还有点兔第三季》草》二次元 行 今 二次元行 w》我也可以啊。。。。》智乃这么主动的啊》》gochiusa》》莲子》铁沙掌》新角色》》nyonmyon》夏米子变攻了啊》耳差分》Clearite》暑》这人魔怔了》rityulate》gggrande》》情色什么，你很成熟 小穴都是粉的》》手艺好精湛 zhan》》太平了》所以这是。。。小花酱cos吼木啦》幼驯染 出》nitutituti》不是有版权吗》文登神器甩棍和喷雾可破之》应该是污染小成本低》》雨》这军舰有点老啊，没有相控阵雷达》unknown》》组 。》突然 妹》》这是百团大战的打法》千夜 。》推 。 惚 。》》天使》食》这个提比不错》看上去还不错》八九寺 可爱》双子》蓝鲸的声音这么大的吗》ne on》久川飒》La na triplepeach》育》镜音》》》五颜六色的》文》摸摸》》》文》不过晚上可以看星星很爽》仲良》苹果糖》》妖梦》井藤 nanaleia》可爱》》》查了一下就是这个 终于知道名字了》netaroulife》不是机甲的》chiri atoranta》》》宿题开始》。》这是kiss的表情》男女平等》这前辈这么大的吗》》》所以可以用这个理由女装》威海漫展 大黄蜂机器人 想要了》情色威海漫展 我想呗coser小姐姐破处》是妹子》大啊 来威海吃把》情色我还是不太懂 为什么女老师 有丁丁》情色威海 威海处男求破处 18 22环翠区 gkd 这奶子好大OVO 下面也很软 AWSL》是的呀》日本的女生也喜欢这样子吗》情色女老师哪来的丁丁》威海漫展 发现硬核cos》情色威海处男 威海买可乐 威海男装大佬女装大佬帅气小哥哥可爱小姐姐 都可以 coser 份 04女装处男求破处 要求最好是150 165 体重45 50KG 环翠区的优先 经区高区会考虑 临港可能会不去 文登荣成乳山可以主动动车来环翠区优先考 虑基本之会在环翠区不会乱走动 路痴一枚 动车房间费用自理 小哥哥不收取费用》抱枕不错》我在说一下 威海 买可乐 威海买可乐 的粉丝或者看到我这条推文的 只要是威海的小姐姐请主动私信我 不然漫展见不到我了哦》情色动漫展人气网红coser二次元必

### 提取每一类的关键词

In [8]:
#获取聚类后的数据
cluster_data=get_cluster_data(twitter_data,num_clusters)

#把聚好类的数据，转换成dataframe数据格式
cluster_dfdata=pd.DataFrame(cluster_data)
cluster_dfdata_T=cluster_dfdata.T #获得矩阵的转置

#使用tf-idf法提取关键词

stopkey = [w.strip() for w in codecs.open('./stopwords.txt', 'r').readlines()]
# tf-idf关键词抽取
result = getKeywords_tfidf(cluster_dfdata_T,stopkey,10)
result

,id,key
0,"[5d68f17d047c01f0a5195a14, 5d68f17d047c01f0a51...",情色 男生 女生 交友 寻约 平台 华人 女神 丝袜 漫展
1,"[5d68f17d047c01f0a5195a16, 5d68f17d047c01f0a51...",示威者 警方 示威 出动 炮车 关系 表示 贸易战 总统 运动
2,"[5d68f178047c01f0a519557b, 5d68f17d047c01f0a51...",魔法 没有 感觉 战斗 设备 少女 遇到 播出 笑话 绝食
3,"[5d68f17d047c01f0a5195a12, 5d68f17d047c01f0a51...",谣言 媒体 抹黑 暴徒 造谣 警察 编造 智商 程度 已经
4,"[5d68f17d047c01f0a5195a28, 5d68f17d047c01f0a51...",地球村 世界 有限公司 诈骗 卫民 政府 公司 成立 资金 制度
5,"[5d68f17d047c01f0a5195a30, 5d68f17d047c01f0a51...",自由 沦陷区 编造 媒体 总统 录音 路透社 并称 手势 潜伏
